### Drosophila melanogaster genes 

To execute bash code we created a snippets through nbextensions containing the following paths in order to avoid copy and paste cell contains. %%bash magic do not recognize previous variables. Adding mkdir command to create necesary paths

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
BASIC='/home/jmurga/mkt/201902/rawData/annotations/basicAnnotation'
CDS='/home/jmurga/mkt/201902/rawData/annotations/cds'
GENES='/home/jmurga/mkt/201902/rawData/annotations/genes'
TEMPORAL='/home/jmurga/mkt/201902/rawData/annotations/tmp'

mkdir -p ${DATA}
mkdir -p ${BASIC}
mkdir -p ${CDS}
mkdir -p ${GENES}
mkdir -p ${TEMPORAL}

Defining global path to python. In this case these variables will be stored after one execution

In [1]:
SRC='/home/jmurga/mkt/201902/scripts/src'
DATA='/home/jmurga/mkt/201902/rawData'

Required libraries

In [2]:
import os
import re
import sys
import numpy as np
import pandas as pd
from numpy import array 
from pyfaidx import Fasta

#### Download Flybase annotation

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'

cd ${DATA}
# Flybase annotation
wget ftp://ftp.flybase.net/genomes/Drosophila_melanogaster/dmel_r5.57_FB2014_03/gff/dmel-all-no-analysis-r5.57.gff.gz
gunzip dmel-all-no-analysis-r5.57.gff.gz

#### Parsing and cleaning flybase annotation to execute all operations by chr
Next operation will be faster due to grep on smaller files. Space is not a problem in our server right now. Each folder contain an specific file foreach chromosome. 

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
BASIC='/home/jmurga/mkt/201902/rawData/annotations/basicAnnotation'
CDS='/home/jmurga/mkt/201902/rawData/annotations/cds'
GENES='/home/jmurga/mkt/201902/rawData/annotations/genes'
TEMPORAL='/home/jmurga/mkt/201902/rawData/annotations/tmp'

mkdir -p ${BASIC}
mkdir -p ${CDS}
mkdir -p ${GENES}
mkdir -p ${TEMPORAL}

# Extract sequences f
# Extract annotations from gff file
sed -e '/^>/,$d' ${DATA}/dmel-all-no-analysis-r5.57.gff | sed -e '/^2LHet/d' -e  '/2RHet/d' -e '/3LHet/d' -e '/3RHet/d' -e  '/^4\t/d' -e '/dmel_mitochondrion_genome/d' -e '/^U\t/d' -e '/^Uextra\t/d' -e  '/^XHet\t/d' -e '/^YHet\t/d' -e'/\tCG/d' > ${DATA}/dmelFiltered.gff
# Extract genes information
grep -P "\tgene\t" ${DATA}/dmelFiltered.gff  > ${DATA}/dmelFilteredGenes.gff
# Extract CDS information. Only protein coding genes
grep -P "\tCDS\t" ${DATA}/dmelFiltered.gff  > ${DATA}/dmelFilteredCds.gff
# Coding gene list. gene_id always on column 9, 3th field
cut -f1,9 ${DATA}/dmelFilteredCds.gff | tr ';' '\t' | cut -f1,2 | sort -u | sort -k1,1 > ${DATA}/codingGeneList.txt


CHR=( 2L 2R 3L 3R X )
# Parse gff file by chr
for chrNumber in "${CHR[@]}"
do
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredGenes.gff | sort -k4,4n > ${BASIC}/gencode.v27lift37.basic.annotation.chr${chrNumber}.gff3 
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredCds.gff | sort -k1,1 -k4,4n > ${CDS}/dmelFilteredCdsChr${chrNumber}.gff3 
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredGenes.gff | sort -k1,1 -k4,4n > ${GENES}/dmelFilteredGenesChr${chrNumber}.gff3 
done

rm ${DATA}/dmel-all-no-analysis-r5.57.gff

#### Basic cleaned gene file
This file will include information about chromosomes, start coordinates, end coordinates strand, gene id and gene name. It will be and perform calculations on gene coordinates. Kind of gff file easier to work with

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
GENES='/home/jmurga/mkt/201902/rawData/annotations/genes'
touch ${DATA}/flybaseGenesCleaned.tab 
printf "chr\tstartGene\tendGene\tstrand\tid\tname\n" > ${DATA}/flybaseGenesCleaned.tab

time while read LINE;
do 
    CHR=$(echo ${LINE} | cut -d' ' -f1)
    GENE=$(echo ${LINE} | cut -d' ' -f2 | sed 's/-cds//g')
    # echo $GENE

    fgrep `echo "${GENE};"`  ${GENES}/dmelFilteredGenesChr${CHR}.gff3 | fgrep ${CHR} |  cut -f1,4,5,7,9 | tr ';' '\t' | cut -f1,2,3,4,5,6 
    
done < ${DATA}/codingGeneList.txt | tr ' ' '\t' | sed 's/ID=//g' | sed 's/Name=//g' | sort -k1,1 -k2,2n >> ${DATA}/flybaseGenesCleaned.tab

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',header = 0,sep='\t')
dfGenes.head()

File to merge features annotated with gene name instead of gene id

In [ ]:
idName = dfGenes[['id','name']]
idName.to_csv(DATA + '/annotations/idName.tab',sep='\t',index=False,header=True)

#### Extracting CDS coordinates from largest transcript by gene
From the largest transcript we recover id, size, and CDS coordinates. Moreover we save number of transcripts too.  
Coordinates will be used to recover CDS sequences and to calcute Derived Allele Frequencies from 0fold and 4fold degenerate sequences

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
CDS='/home/jmurga/mkt/201902/rawData/annotations/cds'
TEMPORAL='/home/jmurga/mkt/201902/rawData/annotations/tmp'

touch ${DATA}/cdsCoordinates.tab
printf "name\tchr\tnumberOfTranscript\ttranscript\ttranscriptSize\tcoordinates\n" > ${DATA}/cdsCoordinates.tab

count=0
time tail -n+2 ${DATA}/flybaseGenesCleaned.tab | while read LINE;
do 
    echo '*************'    
    CHR=$(echo ${LINE} | cut -d' ' -f1)
    GENE=$(echo ${LINE} | cut -d' ' -f6)
    
    printf ${count}
    # fgrep ${GENE} ${CDS}/dmelFilteredCdsChr${CHR}.gff3 | cut -f9 | cut -d';' -f2 | cut -d'=' -f2 | sort -u > ${TEMPORAL}/transcriptTmp.tab
    
    fgrep `echo "Name=${GENE}-cds;"` ${CDS}/dmelFilteredCdsChr${CHR}.gff3 | grep -P "${CHR}\t" | cut -f9 | tr ';' '\n' | fgrep Parent | sort -u | tr ',' '\n' | cut -d'=' -f2 | sort -u > ${TEMPORAL}/transcriptTmp.tab

    largestTranscript=$(while read transcript; do fgrep ${transcript} ${CDS}/dmelFilteredCdsChr${CHR}.gff3 | awk '{print $5-$4}' | awk -v transcript="$transcript"  '{sum+=$1} END{print transcript"\t"sum}' ;done < ${TEMPORAL}/transcriptTmp.tab | sort -nrk2,2 | head -1 )
    
    paste <(echo $GENE) <(echo $CHR) <(wc -l ${TEMPORAL}/transcriptTmp.tab | cut -d' ' -f1) <(echo $largestTranscript | cut -d' ' -f1) <(echo $largestTranscript | cut -d' ' -f2) <(fgrep `echo ${largestTranscript} | cut -d' ' -f1` ${CDS}/dmelFilteredCdsChr${CHR}.gff3  | cut -f4,5 | sort -k1,1n | awk '{printf $1","$2","}' | sed 's/.$//') >> ${DATA}/cdsCoordinates.tab
    
    (( count++ ))

done 


Opening cdsCoordinates file to extract fasta sequence using pyfaidx

In [3]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',header = 0,usecols=['id','name','chr','strand'],sep='\t')

cds = pd.read_csv(DATA + '/annotations/cdsCoordinates.tab',header=0,usecols=['name','chr','coordinates'],sep='\t')

cds = pd.merge(cds, dfGenes,  how='inner', left_on=['chr','name'], right_on = ['chr','name'])

In [ ]:
cds.head()

In [ ]:
cds[cds['chr']=='X'].head(1)

In [ ]:
# Rewrite file each execution
# multifasta = open(DATA + '/annotations/fop/cds.fa', 'w')
baseComplement = {'A':'T','C':'G','G':'C','T':'A','N':'N','-':'-'}

for index, row in cds[cds['chr']=='2L'].iterrows():
# for index, row in cds.iterrows():
    print(index)
    
    # Convert CDS list into numeric array
    coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
    coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
    
    # Open ref and outgroup
    ref = Fasta(DATA + '/fastas/ref/Chr' + row['chr'] +'.fasta')
    outgroup = Fasta(DATA + '/fastas/outgroup/dsim/Simulans_Chr' + row['chr'] +'.seq')
    
    ## Extract ref and outgroup seq
        refSeq = ref.get_spliced_seq(row['chr'], coordinates).seq.upper()
        outgroupSeq = outgroup.get_spliced_seq('dsim' + row['chr'], coordinates).seq.upper()
   
    # Open population multifasta
    popFasta = Fasta(DATA + '/fastas/alignments/RAL/' + 'RAL_' + 'Chr' + row['chr'] +'.seq')
    
    #Extract samples
    samples =chrFile.keys()

    # Open variables and file to write
    multifasta = open(DATA + '/fastas/genes/RAL/' + row['id'] +'.fa', 'w')
    aln = ''
    ## Empty matrix to append ref, pop and outgroup
matrixSequences=np.empty([len(samples)+2,len(refSeq)],dtype='str')
matrixSequences[0] = refSeq
matrixSequences[len(matrixSequences)] = outgroupSeq
    if(row['strand'] == '-'):
#     if((len(refSeq)/3).is_integer()):
        refSeq = refSeq[::-1]
        letters = list(seq)
        letters = [baseComplement[base] for base in letters]
        refSeq = ''.join(letters)
    
        for i in samples:
            tmp = popFasta.get_spliced_seq(i, coordinates).seq.upper()
            tmp = tmp[::-1]

            letters = list(tmp)
            letters = [baseComplement[base] for base in letters]
            tmp = ''.join(letters)
            aln += '>' + i + '\n' + tmp + '\n'
    else:
        for i in samples:
            tmp += popFasta.get_spliced_seq(i, coordinates).seq.upper()
            tmp = tmp[::-1]

            letters = list(tmp)
            letters = [baseComplement[base] for base in letters]
            tmp = ''.join(letters)
            aln += '>' + i + '\n' + tmp + '\n'

